# Pricing Exotic Options with TensorFlow

When i was writing my post about gradient descent and automatic differentiation in TensorFLow I had the idea to use TensorFlow for Monte-Carlo pricing of some path dependent derivates.

TensorFlow supports GPU computing (unfortunatly I can not try this on my laptop) which can speed and with TensorFlows automatic differentiation we can get analytical 'path' derivates. Calculating path derivates with a bump and revaluation is usally very computational costly and can be numerical unstable, depending on the bump size.

In this post we want to focus on the implementation in TensorFlow therefore we will use a Black-Scholes model simplicity and try to price a Plain-Vanilla, a Down-And-Out Barrier and a Bermudan Call Option.

Lets start with the plain vanilla one.

In [1]:
import numpy as np
import tensorflow as tf
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [117]:
## Plain Vanilla Call in TensorFlow

def blackScholes_py(S_0, strike, time_to_expiry, implied_vol, riskfree_rate):
    S = S_0
    K = strike
    dt = time_to_expiry
    sigma = implied_vol
    r = riskfree_rate
    Phi = stats.norm.cdf
    d_1 = (np.log(S_0 / K) + (r+sigma**2/2)*dt) / (sigma*np.sqrt(dt))
    d_2 = d_1 - sigma*np.sqrt(dt)
    return S*Phi(d_1) - K*np.exp(-r*dt)*Phi(d_2)


def blackScholes_tf_pricer():
    # Build the static computational graph
    S = tf.placeholder(tf.float32)
    K = tf.placeholder(tf.float32)
    dt = tf.placeholder(tf.float32)
    sigma = tf.placeholder(tf.float32)
    r = tf.placeholder(tf.float32)
    Phi = tf.distributions.Normal(0.,1.).cdf
    d_1 = (tf.log(S / K) + (r+sigma**2/2)*dt) / (sigma*tf.sqrt(dt))
    d_2 = d_1 - sigma*tf.sqrt(dt)
    npv =  S*Phi(d_1) - K*tf.exp(-r*dt)*Phi(d_2)
    greeks = tf.gradients(npv, [S, sigma, r, K, dt])
    # Calculate mixed 2nd order greeks for S (esp. gamma, vanna) and sigma (esp. volga)
    dS_2ndOrder = tf.gradients(greeks[0], [S, sigma, r, K, dt]) 
    dsigma_2ndOrder = tf.gradients(greeks[1], [S, sigma, r, K, dt]) 
    # Function to feed in the input and calculate the computational graph
    def execute_graph(S_0, strike, time_to_expiry, implied_vol, riskfree_rate):
        with tf.Session() as sess:
            res = sess.run([npv, greeks, dS_2ndOrder, dsigma_2ndOrder], 
                           {
                               S: S_0,
                               K : strike,
                               r : riskfree_rate,
                               sigma: implied_vol,
                               dt: time_to_expiry})
        return res
    return execute_graph
        
    

In [118]:
tf_pricer = blackScholes_tf_pricer()

In [139]:
blackScholes_py(100., 110., 2., 0.2, 0.03) - (100/90)**(1-0.03/0.2**2)*blackScholes_py(90**2/100., 110., 2., 0.2, 0.03)

6.934486895210796

In [120]:
%%timeit
npv = tf_pricer(100., 110., 2., 0.2, 0.03)
npv

2.15 s ± 21.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Monte Carlo with TensorFlow

Lets try a path dependent exotic option, a down and out call. A down-and-out Call behaves as a normal Call but if the price of the underlying touch or fall below a certain level (the barrier B) at any time until the expiry the option, then it becomes worthless, even if its at expiry in the money.

A down and out call is cheaper than the plain vanilla case, since there is a risk that the option get knocked out before reaching the expiry. It can be used to reduce the hedging costs.

In the Black-Scholes model there is again a closed formula to calculate the price. See ... .

We want to price this kind of option in TensorFlow with a Monte-Carlo Simulation and let TensorFLow calculate the path derivates with automatic differentitation.

First implement the analytical solutions in 'pure' Python (actually we rely heavly on numpy) and TensorFLow.

In [167]:
def analytical_downOut_py(S_0, strike, time_to_expiry, implied_vol, riskfree_rate, barrier):
    S = S_0
    K = strike
    dt = time_to_expiry
    sigma = implied_vol
    r = riskfree_rate
    alpha = 0.5 - r/sigma**2
    B = barrier
    Phi = stats.norm.cdf
    d_1 = (np.log(S_0 / K) + (r+sigma**2/2)*dt) / (sigma*np.sqrt(dt))
    d_2 = d_1 - sigma*np.sqrt(dt)
    bs = S*Phi(d_1) - K*np.exp(-r*dt)*Phi(d_2)
    d_1a = (np.log(B**2 / (S*K)) + (r+sigma**2/2)*dt) / (sigma*np.sqrt(dt))
    d_2a = d_1a - sigma*np.sqrt(dt)
    reflection = (S/B)**(1-r/sigma**2) * ((B**2/S)*Phi(d_1a) - K*np.exp(-r*dt)*Phi(d_2a))
    return bs - reflection


def analytical_downOut_tf_pricer(enable_greeks = True):
    S = tf.placeholder(tf.float32)
    K = tf.placeholder(tf.float32)
    dt = tf.placeholder(tf.float32)
    sigma = tf.placeholder(tf.float32)
    r = tf.placeholder(tf.float32)
    B = tf.placeholder(tf.float32)
    Phi = tf.distributions.Normal(0.,1.).cdf
    d_1 = (tf.log(S / K) + (r+sigma**2/2)*dt) / (sigma*tf.sqrt(dt))
    d_2 = d_1 - sigma*tf.sqrt(dt)
    bs_npv =  S*Phi(d_1) - K*tf.exp(-r*dt)*Phi(d_2)
    d_1a = (tf.log(B**2 / (S*K)) + (r+sigma**2/2)*dt) / (sigma*tf.sqrt(dt))
    d_2a = d_1a - sigma*tf.sqrt(dt)
    reflection = (S/B)**(1-r/sigma**2) * ((B**2/S)*Phi(d_1a) - K*tf.exp(-r*dt)*Phi(d_2a))
    npv = bs_npv - reflection
    target_calc = [npv]
    if enable_greeks:
        greeks = tf.gradients(npv, [S, sigma, r, K, dt, B])
        # Calculate mixed 2nd order greeks for S (esp. gamma, vanna) and sigma (esp. volga)
        dS_2ndOrder = tf.gradients(greeks[0], [S, sigma, r, K, dt, B]) 
        dsigma_2ndOrder = tf.gradients(greeks[1], [S, sigma, r, K, dt, B]) 
        # Function to feed in the input and calculate the computational graph
        target_calc += [greeks, dS_2ndOrder, dsigma_2ndOrder]
    def price(S_0, strike, time_to_expiry, implied_vol, riskfree_rate, barrier):
        with tf.Session() as sess:
            
            res = sess.run(target_calc, 
                           {
                               S: S_0,
                               K : strike,
                               r : riskfree_rate,
                               sigma: implied_vol,
                               dt: time_to_expiry,
                               B : barrier})
        return res
    return price

In [501]:
analytical_downOut_py(100., 110., 2., 0.2, 0.03, 90)

6.934486895210796

In [502]:
down_out_pricer = analytical_downOut_tf_pricer(True)

In [503]:
down_out_pricer(100., 110., 2., 0.2, 0.03, 90.)

[6.9344816,
 [0.69351405, 18.208786, 56.060673, -0.22123335, 1.7513491, -0.423125],
 [0.002250306, 1.6649915, 4.7459536, -0.020622324, 0.15443888, 0.02270472],
 [1.6649914, -159.15932, -191.44804, 0.9917286, -6.277489, -2.859783]]

In [321]:
down_out_pricer = analytical_downOut_tf_pricer(False)
%timeit down_out_pricer(100., 110., 2., 0.2, 0.03, 95.)

2.23 s ± 95.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [173]:
down_out_pricer = analytical_downOut_tf_pricer(True)
%timeit down_out_pricer(100., 110., 2., 0.2, 0.03, 90.)

3.85 s ± 54.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [174]:
%timeit analytical_downOut_py(100., 110., 2., 0.2, 0.03, 95.)

265 µs ± 11.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Now lets implement the Monte Carlo Pricing. We will pass the random varibales to the pricing function. 
The pricing function we assume equvidistant timegrid.

In [178]:
def generate_random_variables_for_down_out(steps, samples, seed=42):
    np.random.seed(seed)
    return np.random.randn(samples, steps)

In [398]:
N = generate_random_variables_for_down_out(100, 10000)
N.shape

(10000, 100)

In [399]:
def monte_carlo_down_out_py(S_0, strike, time_to_expiry, implied_vol, riskfree_rate, barrier, stdnorm_random_variates):
    S = S_0
    K = strike
    dt = time_to_expiry / stdnorm_random_variates.shape[1]
    sigma = implied_vol
    r = riskfree_rate
    B = barrier
    # See Advanced Monte Carlo methods for barrier and related exotic options by Emmanuel Gobet
    B_shift = B*np.exp(0.5826*sigma*np.sqrt(dt))
    S_T = S * np.cumprod(np.exp((r-sigma**2/2)*dt+sigma*np.sqrt(dt)*stdnorm_random_variates), axis=1)
    non_touch = (np.min(S_T, axis=1) > B_shift)*1
    call_payout = np.maximum(S_T[:,-1] - K, 0)
    npv = np.mean(non_touch * call_payout)
    return np.exp(-time_to_expiry*r)*npv

In [526]:
%%timeit
monte_carlo_down_out_py(100., 110., 2., 0.2, 0.03, 90., N)

11.5 ms ± 1.55 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [521]:
def monte_carlo_down_out_tf(enable_greeks = True):
    S = tf.placeholder(tf.float32)
    K = tf.placeholder(tf.float32)
    dt = tf.placeholder(tf.float32)
    T = tf.placeholder(tf.float32)
    sigma = tf.placeholder(tf.float32)
    r = tf.placeholder(tf.float32)
    B = tf.placeholder(tf.float32)
    dw = tf.placeholder(tf.float32)
    # See Advanced Monte Carlo methods for barrier and related exotic options by Emmanuel Gobet
    B_shift = B * tf.exp(0.5826*sigma*tf.sqrt(dt))
    S_T = S * tf.cumprod(tf.exp((r-sigma**2/2)*dt+sigma*tf.sqrt(dt)*dw), axis=1)
    non_touch = tf.cast(tf.reduce_all(S_T > B_shift, axis=1), tf.float32)
    call_payout = tf.maximum(S_T[:,-1] - K, 0)
    npv = tf.exp(-r*T) * tf.reduce_mean(non_touch * call_payout)
    target_calc = [npv]
    if enable_greeks:
        greeks = tf.gradients(npv, [S, sigma, r, K, T])
        target_calc += [greeks]
    def pricer(S_0, strike, time_to_expiry, implied_vol, riskfree_rate, barrier, stdnorm_random_variates):
        with tf.Session() as sess:
            timedelta = time_to_expiry / stdnorm_random_variates.shape[1]
            res = sess.run(target_calc, 
                           {
                            S: S_0,
                               K : strike,
                               r : riskfree_rate,
                               sigma: implied_vol,
                               dt : timedelta,
                               T: time_to_expiry,
                               B : barrier,
                               dw : stdnorm_random_variates
                         })
            return res
    return pricer
    

In [522]:
tf_mc_pricer = monte_carlo_down_out_tf()

In [525]:
tf_mc_pricer(100., 110., 2., 0.2, 0.03, 90., N)

[7.035021, [0.31648976, 41.60718, 49.227932, -0.22376387, -0.21105061]]

In [527]:
%%timeit
tf_mc_pricer(100., 110., 2., 0.2, 0.03, 90., N)

3.08 s ± 275 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
